<p align="center">
  <img src="R_container.png" />
</p>


# Introduction
This Kubeflow notebook will first ensure we can connect to the Kubeflow Pipeline Client. Then, it will define a container component called *generate_quote*, create a pipeline called quote_pipeline, and lastly submit the pipeline to the KFP SDK. We will then be able to see the pipeline run and associated artifacts within the Kubeflow Central Dashboard. Run each cell below in order and to view the results.

In [3]:
import kfp
from kfp import dsl
from kfp import compiler
from kfp.client import Client


client = kfp.Client()
print(client.list_experiments())

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Sun, 14 Apr 2024 15:34:06 GMT', 'content-length': '811', 'x-envoy-upstream-service-time': '1', 'server': 'istio-envoy', 'x-envoy-decorator-operation': 'ml-pipeline.kubeflow.svc.cluster.local:8888/*'})
HTTP response body: {"error":"List experiments failed: Failed to authorize with API: Failed to authorize with API: Unauthenticated: Failed to check authorization. User identity is empty in the request header: Unauthenticated: Request header error: there is no user identity header.: Request header error: there is no user identity header.","code":16,"message":"List experiments failed: Failed to authorize with API: Failed to authorize with API: Unauthenticated: Failed to check authorization. User identity is empty in the request header: Unauthenticated: Request header error: there is no user identity header.: Request header error: there is no user identity header.","de

In [4]:
@dsl.container_component
def generate_quote(total_cost: float, state_tax: float, quote: dsl.OutputPath(str)):
    """output a quote"""
    return dsl.ContainerSpec(
        image='chasechristensen/generate_quote:v1',
        command=[
            "Rscript","/usr/local/bin/generate_quote.r"
        ],
        args=[total_cost,state_tax,quote])

In [5]:
@dsl.pipeline
def quote_pipeline(x: float, y: float) -> str:
    task1 = generate_quote(total_cost=x, state_tax=y)
    return task1.output


In [6]:
compiler.Compiler().compile(quote_pipeline, 'pipeline.yml')

In [7]:
run = client.create_run_from_pipeline_package(
    'pipeline.yml',
    arguments={
        'x': 500.0,
        'y':0.35
    },
)

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Sun, 14 Apr 2024 15:34:23 GMT', 'content-length': '811', 'x-envoy-upstream-service-time': '1', 'server': 'istio-envoy', 'x-envoy-decorator-operation': 'ml-pipeline.kubeflow.svc.cluster.local:8888/*'})
HTTP response body: {"error":"List experiments failed: Failed to authorize with API: Failed to authorize with API: Unauthenticated: Failed to check authorization. User identity is empty in the request header: Unauthenticated: Request header error: there is no user identity header.: Request header error: there is no user identity header.","code":16,"message":"List experiments failed: Failed to authorize with API: Failed to authorize with API: Unauthenticated: Failed to check authorization. User identity is empty in the request header: Unauthenticated: Request header error: there is no user identity header.: Request header error: there is no user identity header.","de